In [ ]:
"""from google.colab import drive
drive.mount('/content/drive')"""

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd "/content/drive/MyDrive"

/content/drive/MyDrive


In [ ]:
pip install tensorflow-addons

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.8/611.8 kB 7.5 MB/s eta 0:00:00


In [ ]:
import os
import glob
import pandas as pd
import tensorflow as tf
import numpy as np
import json
import matplotlib.pyplot as plt
import tensorflow_addons as tfa

from PIL import Image, ImageDraw
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate, Dropout, BatchNormalization
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.metrics import Precision, Recall, AUC
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

/usr/local/lib/python3.10/dist-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


In [ ]:
"""
base_folder = 'colab/data/test/dog'
class_folders = os.listdir(base_folder)

image_paths = []
json_paths = []

for class_folder in class_folders:
    image_files = glob.glob(os.path.join(base_folder, class_folder, '*.jpg'))
    json_files = [file.replace('.jpg', '.json') for file in image_files]

    image_paths.extend(image_files)
    json_paths.extend(json_files)

image_paths = [path.replace('\\', '/') for path in image_paths]
json_paths = [path.replace('\\', '/') for path in json_paths]

df = pd.DataFrame({
    'image_path': image_paths,
    'json_path': json_paths,
    'label': [path.split('/')[-2] for path in image_paths]
})"""

"\nbase_folder = 'colab/data/test/dog'\nclass_folders = os.listdir(base_folder)\n\nimage_paths = []\njson_paths = []\n\nfor class_folder in class_folders:\n    image_files = glob.glob(os.path.join(base_folder, class_folder, '*.jpg'))\n    json_files = [file.replace('.jpg', '.json') for file in image_files]\n\n    image_paths.extend(image_files)\n    json_paths.extend(json_files)\n\nimage_paths = [path.replace('\\', '/') for path in image_paths]\njson_paths = [path.replace('\\', '/') for path in json_paths]\n\ndf = pd.DataFrame({\n    'image_path': image_paths,\n    'json_path': json_paths,\n    'label': [path.split('/')[-2] for path in image_paths]\n})"

In [ ]:
"""df.to_csv('colab/csv/df.csv', encoding='utf-8-sig', index=False)"""

"df.to_csv('colab/csv/df.csv', encoding='utf-8-sig', index=False)"

In [ ]:
df = pd.read_csv("colab/csv/df.csv")

In [ ]:
def create_mask(image_path, json_path):
    try:
        with Image.open(image_path) as img:
            original_size = img.size
            img = img.resize((256, 256))
            image = np.array(img, dtype=np.float32)

        with open(json_path, 'r', encoding='UTF-8') as file:
            data = json.load(file)

        mask = Image.new('L', original_size, 0)
        draw = ImageDraw.Draw(mask)

        for annotation in data['labelingInfo']:
            if 'polygon' in annotation:
                polygon = []
                for i in range(1, len(annotation['polygon']['location'][0]) // 2 + 1):
                    x_key = f'x{i}'
                    y_key = f'y{i}'
                    x = annotation['polygon']['location'][0].get(x_key)
                    y = annotation['polygon']['location'][0].get(y_key)
                    if x is not None and y is not None:
                        polygon.append((x, y))

                if polygon:
                    draw.polygon(polygon, outline=1, fill=1)

        mask = mask.resize((256, 256))
        mask = np.array(mask, dtype=np.float32)
        mask = np.expand_dims(mask, axis=-1)

        return image, mask
    except Exception as e:
        empty_image = np.zeros((256, 256, 3), dtype=np.float32)
        empty_mask = np.zeros((256, 256, 1), dtype=np.float32)
        return empty_image, empty_mask

In [ ]:
def rotate_image_and_mask(image, mask, angle):
    image = tfa.image.rotate(image, angle)
    mask = tfa.image.rotate(mask, angle)
    return image, mask

def tf_create_mask(image_path, json_path):
    [image, mask] = tf.numpy_function(create_mask, [image_path, json_path], [tf.float32, tf.float32])
    image.set_shape([256, 256, 3])
    mask.set_shape([256, 256, 1])
    return image, mask

image_paths = df['image_path'].values
json_paths = df['json_path'].values

original_dataset = tf.data.Dataset.from_tensor_slices((image_paths, json_paths))
original_dataset = original_dataset.map(tf_create_mask, num_parallel_calls=tf.data.experimental.AUTOTUNE)

augmented_datasets = []
for angle in [0, np.pi/2, np.pi, 3*np.pi/2]:  # 0, 90, 180, 270도
    augmented_dataset = original_dataset.map(lambda x, y: rotate_image_and_mask(x, y, angle),
                                             num_parallel_calls=tf.data.experimental.AUTOTUNE)
    augmented_datasets.append(augmented_dataset)

# 모든 데이터셋을 결합합니다.
dataset = augmented_datasets[0]
for augmented_dataset in augmented_datasets[1:]:
    dataset = dataset.concatenate(augmented_dataset)

dataset_size = len(image_paths)
train_size = int(0.8 * dataset_size)
val_size = dataset_size - train_size
batch_size = 4

dataset = dataset.shuffle(buffer_size=12000).cache().repeat()
train_dataset = dataset.take(train_size)
val_dataset = dataset.skip(train_size)

train_dataset = train_dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
val_dataset = val_dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

train_dataset = train_dataset.batch(batch_size)
val_dataset = val_dataset.batch(batch_size)

In [ ]:
def unet_model(input_size=(256, 256, 3),dropout_rate=0.5):
    inputs = Input(input_size)

    # 인코더
    conv1 = Conv2D(32, 3, activation='relu', padding='same')(inputs)
    conv1 = BatchNormalization()(conv1)
    conv1 = Conv2D(32, 3, activation='relu', padding='same')(conv1)
    conv1 = BatchNormalization()(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
    pool1 = Dropout(dropout_rate)(pool1)

    conv2 = Conv2D(64, 3, activation='relu', padding='same')(pool1)
    conv2 = BatchNormalization()(conv2)
    conv2 = Conv2D(64, 3, activation='relu', padding='same')(conv2)
    conv2 = BatchNormalization()(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
    pool2 = Dropout(dropout_rate)(pool2)

    conv3 = Conv2D(128, 3, activation='relu', padding='same')(pool2)
    conv3 = BatchNormalization()(conv3)
    conv3 = Conv2D(128, 3, activation='relu', padding='same')(conv3)
    conv3 = BatchNormalization()(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
    pool3 = Dropout(dropout_rate)(pool3)

    conv4 = Conv2D(256, 3, activation='relu', padding='same')(pool3)
    conv4 = BatchNormalization()(conv4)
    conv4 = Conv2D(256, 3, activation='relu', padding='same')(conv4)
    conv4 = BatchNormalization()(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)
    pool4 = Dropout(dropout_rate)(pool4)

    conv5 = Conv2D(512, 3, activation='relu', padding='same')(pool4)
    conv5 = BatchNormalization()(conv5)
    conv5 = Conv2D(512, 3, activation='relu', padding='same')(conv5)
    conv5 = BatchNormalization()(conv5)

    # 디코더
    up6 = UpSampling2D(size=(2, 2))(conv5)
    merge6 = concatenate([conv4, up6], axis=3)
    conv6 = Conv2D(256, 3, activation='relu', padding='same')(merge6)
    conv6 = BatchNormalization()(conv6)
    conv6 = Conv2D(256, 3, activation='relu', padding='same')(conv6)
    conv6 = BatchNormalization()(conv6)
    conv6 = Dropout(dropout_rate)(conv6)

    up7 = UpSampling2D(size=(2, 2))(conv6)
    merge7 = concatenate([conv3, up7], axis=3)
    conv7 = Conv2D(128, 3, activation='relu', padding='same')(merge7)
    conv7 = BatchNormalization()(conv7)
    conv7 = Conv2D(128, 3, activation='relu', padding='same')(conv7)
    conv7 = BatchNormalization()(conv7)
    conv7 = Dropout(dropout_rate)(conv7)

    up8 = UpSampling2D(size=(2, 2))(conv7)
    merge8 = concatenate([conv2, up8], axis=3)
    conv8 = Conv2D(64, 3, activation='relu', padding='same')(merge8)
    conv8 = BatchNormalization()(conv8)
    conv8 = Conv2D(64, 3, activation='relu', padding='same')(conv8)
    conv8 = BatchNormalization()(conv8)
    conv8 = Dropout(dropout_rate)(conv8)

    up9 = UpSampling2D(size=(2, 2))(conv8)
    merge9 = concatenate([conv1, up9], axis=3)
    conv9 = Conv2D(32, 3, activation='relu', padding='same')(merge9)
    conv9 = BatchNormalization()(conv9)
    conv9 = Conv2D(32, 3, activation='relu', padding='same')(conv9)
    conv9 = BatchNormalization()(conv9)
    conv9 = Dropout(dropout_rate)(conv9)

    # 출력
    conv10 = Conv2D(1, (1, 1), activation='sigmoid')(conv9)

    model = Model(inputs=inputs, outputs=conv10)

    return model

class_weight = {0: 1, 1: 40}

unet = unet_model()
unet.compile(optimizer='adam',
             loss=BinaryCrossentropy(from_logits=False),
             metrics=['accuracy', Precision(), Recall(), AUC()])

In [ ]:
early_stopping = EarlyStopping(monitor='val_loss',
                               patience=10,
                               verbose=1,
                               mode='min',
                               restore_best_weights=True)

checkpoint = ModelCheckpoint('colab/model/dog_unet_model_{epoch:02d}.h5',
                             monitor='val_loss',
                             verbose=1,
                             save_best_only=False,
                             mode='min',
                             save_freq='epoch')

In [ ]:
history = unet.fit(train_dataset, epochs=30, steps_per_epoch=train_size // batch_size,
                   validation_data=val_dataset, validation_steps=val_size // batch_size,
                   class_weight=class_weight,
                   callbacks=[checkpoint, early_stopping],
                   verbose=1)

Epoch 1/30
3000/3000 [==============================] - ETA: 0s - loss: 0.6510 - accuracy: 0.8466 - precision: 0.0630 - recall: 0.6371 - auc: 0.8481
Epoch 1: saving model to colab/model/dog_unet_model_01.h5


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


3000/3000 [==============================] - 4816s 1s/step - loss: 0.6510 - accuracy: 0.8466 - precision: 0.0630 - recall: 0.6371 - auc: 0.8481 - val_loss: 0.2443 - val_accuracy: 0.8915 - val_precision: 0.0803 - val_recall: 0.5840 - val_auc: 0.8698
Epoch 2/30
3000/3000 [==============================] - ETA: 0s - loss: 0.5971 - accuracy: 0.8574 - precision: 0.0722 - recall: 0.6819 - auc: 0.8741
Epoch 2: saving model to colab/model/dog_unet_model_02.h5
3000/3000 [==============================] - 858s 249ms/step - loss: 0.5971 - accuracy: 0.8574 - precision: 0.0722 - recall: 0.6819 - auc: 0.8741 - val_loss: 1.1198 - val_accuracy: 0.3648 - val_precision: 0.0223 - val_recall: 0.9834 - val_auc: 0.8448
Epoch 3/30
3000/3000 [==============================] - ETA: 0s - loss: 0.5826 - accuracy: 0.8606 - precision: 0.0741 - recall: 0.6896 - auc: 0.8800
Epoch 3: saving model to colab/model/dog_unet_model_03.h5
3000/3000 [==============================] - 845s 246ms/step - loss: 0.5826 - accuracy

In [ ]:
plt.figure(figsize=(18, 5))

# 훈련 및 검증 손실 그래프
plt.subplot(1, 5, 1)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

# 훈련 및 검증 정확도 그래프
plt.subplot(1, 5, 2)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

# 훈련 및 검증 정밀도 그래프
plt.subplot(1, 5, 3)
plt.plot(history.history['precision_1'], label='Training Precision')
plt.plot(history.history['val_precision_1'], label='Validation Precision')
plt.title('Training and Validation Precision')
plt.xlabel('Epoch')
plt.ylabel('Precision')
plt.legend()

# 훈련 및 검증 재현율 그래프
plt.subplot(1, 5, 4)
plt.plot(history.history['recall_1'], label='Training Recall')
plt.plot(history.history['val_recall_1'], label='Validation Recall')
plt.title('Training and Validation Recall')
plt.xlabel('Epoch')
plt.ylabel('Recall')
plt.legend()

plt.subplot(1, 5, 5)
plt.plot(history.history['auc_1'], label='Training AUC')
plt.plot(history.history['val_auc_1'], label='Validation AUC')
plt.title('Training and Validation AUC')
plt.xlabel('Epoch')
plt.ylabel('AUC')
plt.legend()

plt.tight_layout()
plt.show()